##### Notatki
Tutorial tutaj: https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8

Dane Flickr 8k:
https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip

Polskie podpisy: http://zil.ipipan.waw.pl/Scwad/AIDe?action=AttachFile&do=get&target=AIDe_ANNOTATED_DESCRIPTIONS.zip

In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import tensorflow as tf
from unidecode import unidecode
import csv
from math import ceil

Using TensorFlow backend.


In [2]:
learn_mode = 'lemma_no_pl'

In [3]:
captions_filename = f'../../AIDe_ANNOTATED_DESCRIPTIONS/captions.csv'
tokens_filename = f'../../AIDe_ANNOTATED_DESCRIPTIONS/token_{learn_mode}.txt'
descriptions_filename = f'../descriptions_{learn_mode}.txt'

test_images = []
train_images = []

with open(captions_filename, newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    next(reader)
    with open(tokens_filename, 'w') as ft:
        for row in reader:
            img_name = row[1]
            caption1 = row[2] if not 'no_pl' in learn_mode else unidecode(row[2])
            caption2 = row[3] if not 'no_pl' in learn_mode else unidecode(row[3])
            print(f'{img_name}#0\t{caption1}', file=ft)
            print(f'{img_name}#1\t{caption2}', file=ft)
    

In [4]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load descriptions
doc = load_doc(tokens_filename)
print(doc[:300])

1355703632_5683a4b6fb.jpg#0	Mloda osoba w sportowym stroju i rekawicy robi duzy krok, trzymajac w jednej dloni pilke.
1355703632_5683a4b6fb.jpg#1	Dziewczyna z rekawica na rece biegnie po boisku z zolta pilka w drugiej rece.
2101128963_fdf8b2a0d7.jpg#0	Zawodnik na boisku ze skorzana rekawica robi wyk


In [5]:
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 1000 


In [6]:
list(descriptions.keys())[:5]

['1355703632_5683a4b6fb',
 '2101128963_fdf8b2a0d7',
 '241345522_c3c266a02a',
 '241347460_81d5d62bf6',
 '2521938802_853224f378']

In [7]:
descriptions['1355703632_5683a4b6fb']

['Mloda osoba w sportowym stroju i rekawicy robi duzy krok, trzymajac w jednej dloni pilke.',
 'Dziewczyna z rekawica na rece biegnie po boisku z zolta pilka w drugiej rece.']

In [8]:
descriptions['2521938802_853224f378']

['Mezczyzna w bialym sportowym stroju i czarnym kasku stoi na trawie na zgietych nogach, trzymajac za plecami gruba palke.',
 'Mezczyzna z ochraniaczami na nogach stoi na trawie w rozkroku.']

In [9]:
import spacy
nlp = spacy.load('pl_spacy_model')

def clean_descriptions(descriptions):
  # prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
      desc = desc_list[i]
      doc = nlp(desc)
      # tokenize
      desc = desc.split()
      # convert to lower case
      desc = [word.lower() for word in desc]
# Lematyzacja
#### spacy model from here: http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=view&target=pl_spacy_model-0.1.0.tar.gz
#### spacy dokumentacja - https://spacy.io/usage
      if 'lemma' in learn_mode:
          desc = [word.lemma_ for word in doc]
      # remove punctuation from each token
      desc = [w.translate(table) for w in desc]
      # remove tokens with numbers in them
      desc = [word for word in desc if word.isalpha()]
      # store as string
      desc_list[i] =  ' '.join(desc)

# clean descriptions
clean_descriptions(descriptions)

C:\Users\macie\anaconda3\envs\deep\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'pl_model' (0.1.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.4). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [10]:
descriptions['1355703632_5683a4b6fb']

['mloda osoba w sportowy strój i rekawicy robić duzy krok trzymajac w jeden dloni pilke',
 'dziewczyna z rekawica na rece biec po boisko z zolta pilka w drugi rece']

In [11]:
descriptions['2521938802_853224f378']

['mezczyzna w bialym sportowy strój i czarny kask stać na trawa na zgietych noga trzymajac za plecy gruby palke',
 'mezczyzna z ochraniacz na noga stać na trawa w rozkrok']

In [12]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 2508


In [13]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w', encoding='utf-8')
	file.write(data)
	file.close()

save_descriptions(descriptions, descriptions_filename)

In [14]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

train = []

# load training dataset (6K)
for i in range(0,5): 
    train.append(load_set(f'randomState/trainImages{i}.txt'))
    print('Dataset: %d' % len(train[i]))

Dataset: 700
Dataset: 700
Dataset: 700
Dataset: 700
Dataset: 700


In [15]:
# Below path contains all the images
images = '../../Flickr8k_Dataset/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [16]:
train_images = []
train_img = []

# Read the train image names in a set
for i in range(0,5):
    train_images.append(set(open(f'randomState/trainImages{i}.txt', 'r').read().strip().split('\n')))
    # Create a list of all the training images with their full path names
    train_img.append([])

    for im in img: # img is list of full path names of all images
        if im[len(images):] in train_images[i]: # Check if the image belongs to training set
            train_img[i].append(im) # Add it to the list of train images

In [17]:
test_images = []
test_img = []

# Read the train image names in a set
for i in range(0,5):
    test_images.append(set(open(f'randomState/testImages{i}.txt', 'r').read().strip().split('\n')))

    # Create a list of all the training images with their full path names
    test_img.append([])

    for im in img: # img is list of full path names of all images
        if im[len(images):] in test_images[i]: # Check if the image belongs to training set
            test_img[i].append(im) # Add it to the list of train images

In [18]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

train_descriptions = []

# descriptions
for i in range(0,5):
    train_descriptions.append(load_clean_descriptions(descriptions_filename, train[i]))
    print('Descriptions: train=%d' % len(train_descriptions[i]))

Descriptions: train=700
Descriptions: train=700
Descriptions: train=700
Descriptions: train=700
Descriptions: train=700


{'1355703632_5683a4b6fb': ['startseq mloda osoba w sportowy strĂłj i rekawicy robiÄ‡ duzy krok trzymajac w jeden dloni pilke endseq',
  'startseq dziewczyna z rekawica na rece biec po boisko z zolta pilka w drugi rece endseq'],
 '2101128963_fdf8b2a0d7': ['startseq zawodnik na boisko z skorzana rekawica robiÄ‡ wykrok w przod na piasek endseq',
  'startseq mezczyzna w sportowy strĂłj staÄ‡ na ugietej noga na boisko drugi noge wystawiajac do tyle endseq'],
 '241345522_c3c266a02a': ['startseq zawodnik z pilka biec po boisko i skreca w prawo strone endseq',
  'startseq mezczyzna w sportowy strĂłj i kask biec po boisko z pilka w rek a daleko biec drugi mezczyzna w bluzka w paska endseq'],
 '241347460_81d5d62bf6': ['startseq czlowiek w sportowy strĂłj i kask biec na boisko w kierunek lecacej pilki endseq',
  'startseq podluzna pilka przelatywaÄ‡ przed zawodnik w kask na boisko endseq'],
 '2681215810_00b0642f7b': ['startseq mezczyzna w okulary przeciwslonecznych staÄ‡ na trawa trzymajac w rek 

In [59]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [60]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [61]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [62]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

###### Notatki
Poniższy kod zajmuje bardzo dużo czasu.

In [63]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
encoding_train = []

for i in range(0,5):
    start = time()
    encoding_train.append({})
    for img in train_img[i]:
        encoding_train[i][img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................Time taken in seconds = 93.60391926765442
..................................................................................................................................................................................................................................................................

In [66]:
import os
import pickle


for i in range(0,5):
    writepath = f'Pickle/encoded_train_images_{learn_mode}_{i}.pkl'

    mode = 'a' if os.path.exists(writepath) else 'wb'

    # Save the bottleneck train features to disk
    with open(writepath, "wb") as encoded_pickle:
        pickle.dump(encoding_train[i], encoded_pickle)

In [67]:
# Call the funtion to encode all the test images - Execute this only once
encoding_test = []

for i in range(0,5):
    start = time()
    encoding_test.append({})
    for img in test_img[i]:
        encoding_test[i][img[len(images):]] = encode(img)
        print('.', end='')
    print("Time taken in seconds =", time()-start)

............................................................................................................................................................................................................................................................................................................Time taken in seconds = 39.06138777732849
............................................................................................................................................................................................................................................................................................................Time taken in seconds = 37.48682403564453
............................................................................................................................................................................................................................................................................................................Time taken in se

In [68]:
# Save the bottleneck test features to disk
for i in range(0,5):
    with open(f'Pickle/encoded_test_images_{learn_mode}_{i}.pkl', "wb") as encoded_pickle:
        pickle.dump(encoding_test[i], encoded_pickle)

In [70]:
train_features = []

for i in range(0,5):
    train_features.append(load(open(f'Pickle/encoded_train_images_{learn_mode}_{i}.pkl', "rb")))
    print('Photos: train=%d' % len(train_features[i]))

Photos: train=700
Photos: train=700
Photos: train=700
Photos: train=700
Photos: train=700


In [72]:
# Create a list of all the training captions
all_train_captions = []

for i in range(0,5):
    all_train_captions.append([])
    for key, val in train_descriptions[i].items():
        for cap in val:
            all_train_captions[i].append(cap)
    print(len(all_train_captions[i]))

1400
1400
1400
1400
1400


In [74]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 10
word_counts = []
vocab = []

for i in range(0,5):
    word_counts.append({})
    nsents = 0
    for sent in all_train_captions[i]:
        nsents += 1
        for w in sent.split(' '):
            word_counts[i][w] = word_counts[i].get(w, 0) + 1

    vocab.append([w for w in word_counts[i] if word_counts[i][w] >= word_count_threshold])
    print('preprocessed words %d -> %d' % (len(word_counts[i]), len(vocab[i])))

preprocessed words 2115 -> 236
preprocessed words 2100 -> 244
preprocessed words 2077 -> 245
preprocessed words 2075 -> 245
preprocessed words 2054 -> 244


In [77]:
ixtoword = []
wordtoix = []

for i in range(0,5):
    ixtoword.append({})
    wordtoix.append({})

    ix = 1
    for w in vocab[i]:
        wordtoix[i][w] = ix
        ixtoword[i][ix] = w
        ix += 1

In [78]:
vocab_size = []

for i in range(0,5):
    vocab_size.append(len(ixtoword[i]) + 1) # one for appended 0's

vocab_size

[237, 245, 246, 246, 245]

In [81]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def get_max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

max_length = []

# determine the maximum sequence length
for i in range(0,5):
    max_length.append(get_max_length(train_descriptions[i]))
    print('Description Length: %d' % max_length[i])

Description Length: 31
Description Length: 31
Description Length: 36
Description Length: 36
Description Length: 36


In [100]:
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch, vocab_size):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
#                 yield [[array(X1), array(X2)], array(y)]
                yield ([array(X1), array(X2)], array(y)) 
                X1, X2, y = list(), list(), list()
                n=0

Embedings pobrane stąd: https://github.com/sdadas/polish-nlp-resources/releases/download/v1.0/glove.zip

In [84]:
# Load Glove vectors
glove_dir = '../../glove'
embeddings_index = []

for i in range(0,5):
    embeddings_index.append({}) # empty dictionary

f = open(os.path.join(glove_dir, 'glove_100_3_polish.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    coefs = np.asarray(values[1:], dtype='float32')
    for i in range(0,5):
        word = values[0] if learn_mode != 'no_pl' else unidecode(values[0])
        embeddings_index[i][word] = coefs


In [85]:
embedding_dim = 100
embedding_matrix = []

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
for i in range(0,5):
    embedding_matrix.append(np.zeros((vocab_size[i], embedding_dim)))

    for word, j in wordtoix[i].items():
        #if i < max_words:
        embedding_vector = embeddings_index[i].get(word)
        if embedding_vector is not None:
            # Words not found in the embedding index will be all zeros
            embedding_matrix[i][j] = embedding_vector

In [91]:
model = []

for i in range(0,5):
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    inputs2 = Input(shape=(max_length[i],))
    se1 = Embedding(vocab_size[i], embedding_dim, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size[i], activation='softmax')(decoder2)
    model.append(Model(inputs=[inputs1, inputs2], outputs=outputs))

In [93]:
for i in range(0,5):
    model[i].layers[2].set_weights([embedding_matrix[i]])
    model[i].layers[2].trainable = False

In [94]:
for i in range(0,5):
    model[i].compile(loss='categorical_crossentropy', optimizer='adam')

In [95]:
epochs = 10
number_pics_per_bath = 3
steps = []
for i in range(0,5):
    steps.append(len(train_descriptions[i])//number_pics_per_bath)

Kolejny dość czasochłonny krok

In [102]:
for i in range(0,5):
    for epoch in range(epochs):
        generator = data_generator(train_descriptions[i], train_features[i], wordtoix[i], max_length[i], number_pics_per_bath, vocab_size[i])
        print(generator)
        model[i].fit_generator(generator, epochs=1, steps_per_epoch=steps[i], verbose=1)
        writepath = f'model_weights/model_{str(epoch)}_{learn_mode}_{i}.h5'

        mode = 'a' if os.path.exists(writepath) else 'wb'
    #     file = open(writepath, mode)
        model[i].save(writepath)

<generator object data_generator at 0x00000176597BFF48>
Epoch 1/1
233/233 [==============================] - 21s 91ms/step - loss: 3.9051
<generator object data_generator at 0x00000176597BFE48>
Epoch 1/1
233/233 [==============================] - 21s 91ms/step - loss: 3.6877
<generator object data_generator at 0x0000017670FFE2C8>
Epoch 1/1
233/233 [==============================] - 22s 96ms/step - loss: 3.5096
<generator object data_generator at 0x00000176597BFF48>
Epoch 1/1
233/233 [==============================] - 23s 98ms/step - loss: 3.3352
<generator object data_generator at 0x0000017670FEAE48>
Epoch 1/1
233/233 [==============================] - 22s 96ms/step - loss: 3.2169
<generator object data_generator at 0x00000176597BFDC8>
Epoch 1/1
233/233 [==============================] - 22s 94ms/step - loss: 3.1065
<generator object data_generator at 0x0000017670FEAE48>
Epoch 1/1
233/233 [==============================] - 22s 96ms/step - loss: 2.9848
<generator object data_generator a

In [103]:
for i in range(0,5):
    for epoch in range(epochs):
        generator = data_generator(train_descriptions[i], train_features[i], wordtoix[i], max_length[i], number_pics_per_bath, vocab_size[i])
        model[i].fit_generator(generator, epochs=1, steps_per_epoch=steps[i], verbose=1)
        model[i].save(f'model_weights/model_{str(epoch)}_{learn_mode}_{i}.h5')

Epoch 1/1
233/233 [==============================] - 20s 87ms/step - loss: 2.5787
Epoch 1/1
233/233 [==============================] - 20s 88ms/step - loss: 2.4982
Epoch 1/1
233/233 [==============================] - 20s 88ms/step - loss: 2.4198
Epoch 1/1
233/233 [==============================] - 20s 87ms/step - loss: 2.3317
Epoch 1/1
233/233 [==============================] - 20s 86ms/step - loss: 2.2633
Epoch 1/1
233/233 [==============================] - 21s 89ms/step - loss: 2.1467
Epoch 1/1
233/233 [==============================] - 21s 89ms/step - loss: 2.1354
Epoch 1/1
233/233 [==============================] - 20s 87ms/step - loss: 2.0096
Epoch 1/1
233/233 [==============================] - 20s 87ms/step - loss: 1.9092
Epoch 1/1
233/233 [==============================] - 21s 89ms/step - loss: 1.8395
Epoch 1/1
233/233 [==============================] - 20s 88ms/step - loss: 2.6236
Epoch 1/1
233/233 [==============================] - 20s 87ms/step - loss: 2.5291
Epoch 1/1
233/23

In [105]:
# model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_bath = 6
steps = []
for i in range(0,5):
    steps.append(len(train_descriptions[i])//number_pics_per_bath)

In [106]:
for i in range(0,5):
    for epoch in range(epochs):
        generator = data_generator(train_descriptions[i], train_features[i], wordtoix[i], max_length[i], number_pics_per_bath, vocab_size[i])
        model[i].fit_generator(generator, epochs=1, steps_per_epoch=steps[i], verbose=1)

Epoch 1/1
116/116 [==============================] - 15s 130ms/step - loss: 1.9338
Epoch 1/1
116/116 [==============================] - 14s 125ms/step - loss: 1.8636
Epoch 1/1
116/116 [==============================] - 14s 125ms/step - loss: 1.7987
Epoch 1/1
116/116 [==============================] - 14s 124ms/step - loss: 1.7371
Epoch 1/1
116/116 [==============================] - 15s 125ms/step - loss: 1.6998
Epoch 1/1
116/116 [==============================] - 15s 125ms/step - loss: 1.6124
Epoch 1/1
116/116 [==============================] - 15s 126ms/step - loss: 1.5457
Epoch 1/1
116/116 [==============================] - 15s 126ms/step - loss: 1.4812
Epoch 1/1
116/116 [==============================] - 14s 124ms/step - loss: 1.4246
Epoch 1/1
116/116 [==============================] - 14s 124ms/step - loss: 1.3701
Epoch 1/1
116/116 [==============================] - 15s 127ms/step - loss: 2.0300
Epoch 1/1
116/116 [==============================] - 14s 125ms/step - loss: 1.9574
Epoc

In [108]:
for i in range(0,5):
    model[i].save_weights(f'model_weights/model_30_{learn_mode}_{i}.h5')

In [109]:
for i in range(0,5):
    model[i].load_weights(f'model_weights/model_30_{learn_mode}_{i}.h5')